In [2]:
import osrm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path


# TEST

In [5]:
%%time
# Test (...klappt natürlich nur, wenn die Koordinaten auch im entsprechenden Gebiet für das der Server gerade läuft (!) liegen)
list_coord = [[7.6471, 48.2234],
              [10.3469, 52.0063],
              [10.3531, 52.0383]]

list_id = ['name1', 'name2', 'name3']

time_matrix, snapped_coords, _ = osrm.table(list_coord, ids_origin=list_id, output='dataframe')

time_matrix/60

CPU times: user 4.21 ms, sys: 58 µs, total: 4.27 ms
Wall time: 499 ms


,name1,name2,name3
name1,0.000000,6882.850000,6904.870
name2,6882.995000,0.000000,66.355
name3,6905.273333,66.753333,0.000


In [3]:
wenkerDR = pd.read_csv('DR/Wenker_places_wgs_DR.csv', index_col=0)
wenkerDR.rename(columns={'coords.x1': 'X', 'coords.x2': 'Y'}, inplace=True)

# Alle Duplikate
#duplicates = wenkerDR[wenkerDR.duplicated(subset=['gid', 'ort', 'X', 'Y'], keep=False)]
#print(duplicates.equals(wenkerDR[wenkerDR.duplicated(subset=['gid'], keep=False)]))
#duplicates.sort_values('gid')
# Ohne duplikate (behält das erste Vorkommen eines Duplikates)
wenkerDR_no_dups = wenkerDR[~wenkerDR.duplicated(subset=['gid'])]
gids = wenkerDR_no_dups['gid']
wenkerDR_no_dups_XY = wenkerDR_no_dups[['X', 'Y']]
#gids.to_csv(f"./DR/Distanzmatrix/header_gids.csv", header=False, index=False)

In [7]:
# Falls man nicht alle nehmen möchte (zum Testen o.Ä.)
#wenkerDR_no_dups_XY = wenkerDR_no_dups_XY.iloc[0:500]
#gids = gids.iloc[0:500]

In [8]:
def DR_quadrant(ij):
    i = ij[0]
    j = ij[1]
    orig_start = i*1000
    orig_end   = (i+1)*1000
    dest_start = j*1000
    dest_end   = (j+1)*1000
    time_matrix, snapped_coords, _ = osrm.table(coords_src = wenkerDR_no_dups_XY[orig_start:orig_end].to_numpy().tolist(),
                                                coords_dest=wenkerDR_no_dups_XY[dest_start:dest_end].to_numpy().tolist(),
                                                ids_origin=gids[orig_start:orig_end].to_numpy().tolist(),
                                                ids_dest=gids[dest_start:dest_end].to_numpy().tolist(),
                                                output='df')

    time_matrix.to_csv(f"./DR/Distanzmatrix/{i}_{j}.csv.csv", header=False, index=False)
    print(f"qudrant {i}, {j} done.")

    return (i,j)

In [11]:
%%time
indices = [(i,j) for i in range(len(wenkerDR_no_dups_XY)//1000 +1)
                 for j in range(len(wenkerDR_no_dups_XY)//1000 +1)]
with ThreadPoolExecutor(max_workers=100) as ex:
    for (i,j) in ex.map(DR_quadrant, indices):
        print(f"qudrant {i}, {j} done.")

qudrant 0, 36 done.
qudrant 1, 2 done.
qudrant 0, 37 done.
qudrant 1, 4 done.
qudrant 0, 5 done.
qudrant 1, 7 done.
qudrant 0, 18 done.
qudrant 0, 32 done.
qudrant 1, 15 done.
qudrant 0, 35 done.
qudrant 1, 12 done.
qudrant 0, 14 done.
qudrant 0, 10 done.
qudrant 0, 16 done.
qudrant 0, 15 done.
qudrant 1, 17 done.
qudrant 1, 11 done.
qudrant 1, 10 done.
qudrant 0, 51 done.
qudrant 0, 38 done.
qudrant 0, 48 done.
qudrant 0, 21 done.
qudrant 0, 50 done.
qudrant 0, 41 done.
qudrant 0, 0 done.
qudrant 0, 0 done.
qudrant 0, 17 done.
qudrant 0, 33 done.
qudrant 1, 5 done.
qudrant 1, 14 done.
qudrant 0, 11 done.
qudrant 0, 44 done.
qudrant 1, 6 done.
qudrant 0, 47 done.
qudrant 1, 8 done.
qudrant 0, 28 done.
qudrant 1, 9 done.
qudrant 0, 27 done.
qudrant 0, 49 done.
qudrant 0, 29 done.
qudrant 0, 3 done.
qudrant 0, 22 done.
qudrant 1, 19 done.
qudrant 1, 0 done.
qudrant 0, 9 done.
qudrant 0, 20 done.
qudrant 1, 20 done.
qudrant 1, 36 done.
qudrant 1, 29 done.
qudrant 1, 33 done.
qudrant 1, 32

In [7]:
%%time
# as rows: 2min 49s
# as cols: 2min 2s
MAIN = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/")
DATA_DIR = MAIN/"quadrant_chunks"
OUT_DIR = MAIN/ "rows_as_series" #"as_rows"

gids = pd.read_csv(MAIN/"header_gids.csv", header=None).squeeze()

for i in range(52):
    akk = pd.read_csv(DATA_DIR/f"{i}_0.csv.csv", header=None)
    for j in range(1,52):
        next = pd.read_csv(DATA_DIR/f"{i}_{j}.csv.csv", header=None)
        akk = pd.concat([akk, next], axis=1)
    for k in range(len(akk)):
        pd.DataFrame(akk.iloc[k]).squeeze().to_csv(OUT_DIR/f"{gids[i*1000+k]}.csv", header=False, index=False)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
CPU times: user 1h 31min 40s, sys: 13min 48s, total: 1h 45min 29s
Wall time: 1h 50min 40s


In [4]:
len(gids)

51537

In [5]:
DATA_DIR = Path("/media/praisethesun/INTENSO/DistMatProj/DR/Distanzmatrix/")
OUT_DIR = DATA_DIR/ "as_rows"
gids = pd.read_csv(DATA_DIR/"header_gids.csv", header=None).squeeze()

In [23]:
test_gids = [542008, 97450]
idx = gids[gids == test_gids[1]].index[0]
row = pd.read_csv(OUT_DIR/f"{test_gids[0]}.csv", header=None).squeeze()
row[idx]/60**2

257.01152777777776

In [18]:
for g in test_gids:


382303.7

In [33]:
gids[gids.isin(test_gids)].index.tolist()

[2088, 14310]

In [14]:
wenker_medium = pd.read_csv('medium_AOI/Wenker_places_medium_AOI.csv')
#wenkerDR.rename(columns={'coords.x1': 'X', 'coords.x2': 'Y'}, inplace=True)

# Alle Duplikate
#duplicates = wenkerDR[wenkerDR.duplicated(subset=['gid', 'ort', 'X', 'Y'], keep=False)]
#print(duplicates.equals(wenkerDR[wenkerDR.duplicated(subset=['gid'], keep=False)]))
#duplicates.sort_values('gid')
# Ohne duplikate (behält das erste Vorkommen eines Duplikates)
wenker_medium_no_dups = wenker_medium[~wenker_medium.duplicated(subset=['gid'])]
gids = wenker_medium_no_dups['gid']
wenker_medium_no_dups_XY = wenker_medium_no_dups[['X', 'Y']]
#gids.to_csv(f"./DR/Distanzmatrix/header_gids.csv", header=False, index=False)

In [16]:
def task_medium(i):
    print(f"row({i}) started")
    time_matrix, snapped_coords, _ = osrm.table(coords_src = wenker_medium_no_dups_XY.iloc[i:i+1].to_numpy().tolist(),
                                                coords_dest=wenker_medium_no_dups_XY.to_numpy().tolist(),
                                                ids_origin=gids.iloc[i:i+1].to_numpy().tolist(),
                                                ids_dest=gids.to_numpy().tolist(),
                                                output='df')
    print(f"row({i}) ready")
    time_matrix.to_csv(f"./medium_AOI/Distanzmatrix/{time_matrix.index[0]}.csv", header=False, index=False)
    print(f"row({i}) printed.")

    return i

In [17]:
%%time
with ThreadPoolExecutor(max_workers=64) as ex:
    for i in ex.map(task_medium, list(range(0, 2))):
        print(f"row({i}) done.")

row(0) started
row(1) started
row(1) ready
row(1) printed.
row(0) ready
row(0) printed.
row(0) done.
row(1) done.
CPU times: user 721 ms, sys: 137 ms, total: 858 ms
Wall time: 24min 5s


In [ ]:
time_matrix, snapped_coords, _ = osrm.table(wenker_medium_no_dups_XY.to_numpy(), ids_origin=gids.to_numpy().tolist(), output='dataframe')

In [ ]:
def task_medium_reverse(i):
    print(f"row({i}) started")
    time_matrix, snapped_coords, _ = osrm.table(coords_src = wenker_medium_no_dups_XY.to_numpy().tolist(),
                                                coords_dest=wenker_medium_no_dups_XY.iloc[i:i+1].to_numpy().tolist(),
                                                ids_origin=gids.to_numpy().tolist(),
                                                ids_dest=gids.iloc[i:i+1].to_numpy().tolist(),
                                                output='df')
    print(f"row({i}) ready")
    time_matrix.to_csv(f"./medium_AOI/Distanzmatrix/{time_matrix.index[0]}.csv", header=False, index=False)
    print(f"row({i}) printed.")

    return i